In [1]:
#credit for most of this code: https://medium.com/swlh/how-to-build-a-url-crawler-to-map-a-website-using-python-3e7db83feb7a
from bs4 import BeautifulSoup
import requests
import requests.exceptions
from urllib.parse import urlsplit
from urllib.parse import urlparse
from collections import deque
import re
import lxml
import time
research_articles = []

url = "https://badgerherald.com/"
# a queue of urls to be crawled
new_urls = deque([url])

# a set of urls that we have already been processed 
processed_urls = set()
# a set of domains inside the target website
local_urls = set()
# a set of domains outside the target website
foreign_urls = set()
# a set of broken urls
broken_urls = set()

# process urls one by one until we exhaust the queue
while len(new_urls):
    # move next url from the queue to the set of processed urls
    url = new_urls.popleft()
    processed_urls.add(url)
    # get url's content
    print("Processing %s" % url)
    try:
        response = requests.get(url)
    except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError, requests.exceptions.InvalidURL, requests.exceptions.InvalidSchema):
        # add broken urls to it's own set, then continue
        broken_urls.add(url)
        continue
    time.sleep(1)
    # extract base url to resolve relative links
    parts = urlsplit(url)
    base = "{0.netloc}".format(parts)
    strip_base = base.replace("www.", "")
    base_url = "{0.scheme}://{0.netloc}".format(parts)
    path = url[:url.rfind('/')+1] if '/' in parts.path else url

    # create a beutiful soup for the html document
    soup = BeautifulSoup(response.text, "lxml")
    
    ###########origingla code#############
    key_words = ["sexual assault", "sexual misconduct", "rape", "sexual harassment", "sexual abuse"]
    for phrase in key_words:
        for element in soup.find_all("p"):
            if phrase in element.text and (url not in research_articles):
                research_articles.append(url)
                print("ping")
        for element in soup.find_all("h2"):
            if phrase in element.text and (url not in research_articles):
                research_articles.append(url)
                print("ping")
        for element in soup.find_all("h1"):
            if phrase in element.text and (url not in research_articles):
                research_articles.append(url)
                print("ping")

    ######################################
    for link in soup.find_all('a'):
        # extract link url from the anchor
        anchor = link.attrs["href"] if "href" in link.attrs else ''
        if anchor.startswith('/'):
            local_link = base_url + anchor
            local_urls.add(local_link)
        elif strip_base in anchor:
            local_urls.add(anchor)
        elif not anchor.startswith('http'):
            local_link = path + anchor
            local_urls.add(local_link)
        else:
            foreign_urls.add(anchor)

        for i in local_urls:
            if not i in new_urls and not i in processed_urls:
                if "banter" not in i:
                    new_urls.append(i)

print("finished: " + str(len(processed_urls)))

Processing https://badgerherald.com/tag/rape-kit/
ping
Processing https://badgerherald.com/features/2022/02/01/not-enough-not-yet-uw-makes-progress-on-sexual-assault-response-but-shortcomings-persist/
ping
Processing https://badgerherald.com/opinion/2019/03/28/the-backlog-of-rape-kits-has-shrunk-but-the-fight-to-end-delayed-evidence-testing-rages-on/
ping
Processing https://badgerherald.com/opinion/2018/09/25/the-difference-between-learning-from-mistakes-and-wasting-time/
ping
Processing https://badgerherald.com/./news/
Processing https://badgerherald.com/./features/
ping
Processing https://badgerherald.com/./sports/
Processing https://badgerherald.com/./opinion/


KeyboardInterrupt: 

In [ ]:
print(len(research_articles))